In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [8]:
pip install tweepy

     |██████████████████████████████▋ | 61 kB 5.4 MB/s eta 0:00:011     |████████████████████████████████| 64 kB 1.5 MB/s 
Note: you may need to restart the kernel to use updated packages.


In [9]:
def print_full(x):
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    pd.set_option('display.width', 2000)
    pd.set_option('display.float_format', '{:20,.2f}'.format)
    pd.set_option('display.max_colwidth', None)
    print(x)
    pd.reset_option('display.max_rows')
    pd.reset_option('display.max_columns')
    pd.reset_option('display.width')
    pd.reset_option('display.float_format')
    pd.reset_option('display.max_colwidth')

In [10]:
import re
import string

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.calibration import CalibratedClassifierCV
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.svm import LinearSVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC

from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels
from sklearn.feature_selection import SelectFromModel

from imblearn.pipeline import Pipeline
import pickle


import spacy
nlp = spacy.load("en_core_web_lg")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [11]:
import re
import string
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob
import joblib
from joblib import load
from sklearn.base import TransformerMixin, BaseEstimator

## Load Pipeline

In [12]:
# Text Proccessing

class TextPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, text_attribute):
        self.text_attribute = text_attribute
    
    def fit(self, X, y=None):
        return self
        
    def transform(self, X, *_):
        X_copy = X.copy()
        return X_copy[self.text_attribute].apply(self._preprocess_text)
    
    def _preprocess_text(self, text):
        return self._lemmatize(self._leave_letters_only(self._clean(text)))
    
    def _clean(self, text):
        bad_symbols = '!"#%&\'*+,-<=>?[\\]^_`{|}~'
        text_without_symbols = text.translate(str.maketrans('', '', bad_symbols))

        text_without_bad_words = ''
        for line in text_without_symbols.split('\n'):
            if not line.lower().startswith('from:') and not line.lower().endswith('writes:'):
                text_without_bad_words += line + '\n'

        clean_text = text_without_bad_words
        email_regex = r'([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
        regexes_to_remove = [email_regex, r'Subject:', r'Re:']
        for r in regexes_to_remove:
            clean_text = re.sub(r, '', clean_text)

        return clean_text
    
    def _leave_letters_only(self, text):
        text_without_punctuation = text.translate(str.maketrans('', '', string.punctuation))
        return ' '.join(re.findall("[a-zA-Z]+", text_without_punctuation))
    
    def _lemmatize(self, text):
        doc = nlp(text)
        words = [x.lemma_ for x in [y for y in doc if not y.is_stop and y.pos_ != 'PUNCT' 
                                    and y.pos_ != 'PART' and y.pos_ != 'X']]
        return ' '.join(words)
    

class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [13]:
# load the saved pipleine model
pipeline = load("../input/suicide-text-pipeline2/Suicide_text_classification (1).joblib")

In [14]:
#Twitter client for connecting to Twitter, fetch tweets and get sentiment

class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'MnHwg64FVmmx3Q0DiVVBWsyBU'
        consumer_secret = 'VLVBq928YFQieiCHnsIC3mHdxmfJN798UaAga8S5I31kHje74C'
        access_token = '336239701-BTw0aMAD55Buqf30fAPwSuq0P5OIx2KYHFfIAGgr'
        access_token_secret = 'r3SwjizRcMIa5m83kWUbKYiF2Hbs8HKVlaXl01SoZf6xn'

        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")

    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def get_tweets(self, query, count, lang, locale):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []

        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search_tweets(q = query, count = count, lang = lang, locale = locale)

            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}

                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)

                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)

            # return parsed tweets
            return tweets

        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))


In [24]:
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    global tweets
    tweets = api.get_tweets(query = 'nigeria', count = 200, lang = "en", locale = "ja")

    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(round(100*len(ptweets)/len(tweets),2)))
    "\n"
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(round(100*len(ntweets)/len(tweets), 2)))
    "\n"
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} % \
        ".format(100*(len(tweets) -(len( ntweets )+len( ptweets)))/len(tweets)))

    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    pos = pd.DataFrame(ptweets)
    print(pos)
    """"for tweet in ptweets[:10]:
        print(tweet['text'], "\n")"""

    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    neg = pd.DataFrame(ntweets)
    print(neg)
    """"for tweet in ntweets[:10]:
        print(tweet['text'], "\n")"""

if __name__ == "__main__":
    # calling main function
    main()


Positive tweets percentage: 42.68 %
Negative tweets percentage: 7.32 %
Neutral tweets percentage: 50.0 %         


Positive tweets:
                                                 text sentiment
0   RT @DrawsLa: Hi #PortfolioDay , My name is L.A...  positive
1   RT @SunnyEzeobi: @helper372165  Hello leave @s...  positive
2   @xxyettymamaxx If dem try am.... me sef go joi...  positive
3   RT @CEmogor: 📣New paper: Nigeria is central to...  positive
4   RT @adamash411: Kindly watch the video of Dead...  positive
5   RT @Ambrosia_Ijebu: People left VERY high payi...  positive
6   RT @laralogan: My heart sank when I received t...  positive
7   RT @ireteeh: If you are currently doing Cybers...  positive
8   RT @TheEnike: These are your “law enforcement”...  positive
9   @ivancedric_a Nope! I say my full name as it i...  positive
10  RT @DejiAdesogan: Awesome; I was actually reco...  positive
11  Kano State Ministry of Budget and Planning, an...  positive
12  RT @Diiccey: *Nigeria skit make

In [ ]:
./Suicide_text_classification.joblib

In [25]:
tw = pd.DataFrame(tweets)
tw

text sentiment
0   RT @DrawsLa: Hi #PortfolioDay , My name is L.A...  positive
1   RT @SunnyEzeobi: @helper372165  Hello leave @s...  positive
2   RT @Dracos_2_Brazy: @FavorGrace90 @Respiratory...   neutral
3   @xxyettymamaxx If dem try am.... me sef go joi...  positive
4   Is there a conspiracy to allow ISIS and ISWAP ...   neutral
..                                                ...       ...
77  How low-code development can help small busine...  negative
78  RT @2muchdrag: I stopped watching Nollywood fi...   neutral
79  RT @BigBroTea: I'm emotional because those car...  positive
80  The outside Nigeria that you are, you can't ta...   neutral
81  @MajorPapJ @sylensa_adolf U miss the point but...  positive

[82 rows x 2 columns]

In [26]:
tw_text = tw.drop(columns = ["sentiment"])

In [27]:
pred = pipeline.predict(tw_text)
pred

array(['non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'suicide', 'suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'suicide', 'non-suicide', 'suicide', 'non-suicide', 'suicide',
       'non-suicide', 'suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'suicide', 'suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'non-suicide', 'non-suicide', 'non-suicide', 'non-suicide',
       'suicide', 'non-suicide', 'non-suicide', 'non-suic

In [28]:
pred.shape

(82,)

In [29]:
tw["Prediction"] = pred
print_full(tw.head(50))

                                                                                                                                                          text sentiment   Prediction
0                 RT @DrawsLa: Hi #PortfolioDay , My name is L.A, I’m a concept artist and illustrator from Lagos, Nigeria. I enjoy creating fantasy landscap…  positive  non-suicide
1                 RT @SunnyEzeobi: @helper372165  Hello leave @simon_ekpa out of this, Simon  doing the best work he can do he never differentiated his compl…  positive  non-suicide
2                           RT @Dracos_2_Brazy: @FavorGrace90 @RespiratoryDafe Relationships in Nigeria is a poverty alleviation program, that’s how it’s been   neutral  non-suicide
3                 @xxyettymamaxx If dem try am.... me sef go join the sọ̀rọ̀sókè geng... 🤣🤣🤣\nIndeed many freebies abound in Nigeria.… https://t.co/DOKlZQp4OR  positive  non-suicide
4                 Is there a conspiracy to allow ISIS and ISWAP take over some parts of co